IMPORTING NECESSARY LIBRARIES

In [34]:
import numpy as np
import cv2
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras import layers, models
from sklearn.preprocessing import LabelEncoder


DATA SET LOADING AND PREPROCESSING

In [35]:
# Step 3.2.1: Load and preprocess the dataset
def load_dataset(data_dir, image_size=(224, 224)):
    images = []
    labels = []
    for category in os.listdir(data_dir):
        category_dir = os.path.join(data_dir, category)
        if os.path.isdir(category_dir):
            for filename in os.listdir(category_dir):
                if filename.endswith(".jpg"):
                    img_path = os.path.join(category_dir, filename)
                    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                    img = cv2.equalizeHist(img)  # Apply histogram equalization
                    img = cv2.resize(img, image_size)  # Resize images to a fixed size
                    images.append(img)
                    labels.append(category)
                    print(f"Loaded image: {img_path}, Label: {category}, Shape: {img.shape}")
    return np.array(images), np.array(labels)

SHOWING THE NUMBER OF SAMPLES PRESENT IN DATASET

In [36]:
data_dir = 'C:/Users/Maimoona Sabir/Desktop/CV'  
X, y = load_dataset(data_dir)

# Print the number of samples in the dataset
print(f"Number of samples: {len(X)}")

Loaded image: C:/Users/Maimoona Sabir/Desktop/CV\Buildings2\b1.jpg, Label: Buildings2, Shape: (224, 224)
Loaded image: C:/Users/Maimoona Sabir/Desktop/CV\Buildings2\b11.jpg, Label: Buildings2, Shape: (224, 224)
Loaded image: C:/Users/Maimoona Sabir/Desktop/CV\Buildings2\b12.jpg, Label: Buildings2, Shape: (224, 224)
Loaded image: C:/Users/Maimoona Sabir/Desktop/CV\Buildings2\b13.jpg, Label: Buildings2, Shape: (224, 224)
Loaded image: C:/Users/Maimoona Sabir/Desktop/CV\Buildings2\b14.jpg, Label: Buildings2, Shape: (224, 224)
Loaded image: C:/Users/Maimoona Sabir/Desktop/CV\Buildings2\b15.jpg, Label: Buildings2, Shape: (224, 224)
Loaded image: C:/Users/Maimoona Sabir/Desktop/CV\Buildings2\b2.jpg, Label: Buildings2, Shape: (224, 224)
Loaded image: C:/Users/Maimoona Sabir/Desktop/CV\Buildings2\b3.jpg, Label: Buildings2, Shape: (224, 224)
Loaded image: C:/Users/Maimoona Sabir/Desktop/CV\Buildings2\b4.jpg, Label: Buildings2, Shape: (224, 224)
Loaded image: C:/Users/Maimoona Sabir/Desktop/CV\B

BULDING AND TRAINING THE CNN MODEL

In [42]:
# Check if the dataset directory exists
if os.path.exists(data_dir) and len(os.listdir(data_dir)) > 0:
    # Load and preprocess the dataset
    X, y = load_dataset(data_dir)

    # Check if dataset contains enough samples
    if len(X) > 0:
        # Split the dataset into training and test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # Reshape X_train and X_test to add the channel dimension
        X_train = np.expand_dims(X_train, axis=-1)  # Assuming grayscale images
        X_test = np.expand_dims(X_test, axis=-1)    # Assuming grayscale images
        
        # Convert labels to numerical format using LabelEncoder
        label_encoder = LabelEncoder()
        y_train_encoded = label_encoder.fit_transform(y_train)
        y_test_encoded = label_encoder.transform(y_test)

        # Apply data augmentation
        train_datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest'
        )
        
        train_generator = train_datagen.flow(X_train, y_train_encoded, batch_size=32)

        # Build and compile the model
        model = models.Sequential([
            layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 1)),  # Adjust input shape to accept single-channel images
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(128, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(128, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Flatten(),
            layers.Dropout(0.5),
            layers.Dense(512, activation='relu'),
            layers.Dense(4, activation='softmax')
        ])

        model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

        # Train the model with data generator
        history = model.fit(train_generator, 
                            steps_per_epoch=len(X_train) // 32, 
                            epochs=6, 
                            validation_data=(X_test, y_test_encoded))

        # Evaluate the model
        test_loss, test_accuracy = model.evaluate(X_test, y_test_encoded)
        print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

        # Generate predictions
        y_pred = np.argmax(model.predict(X_test), axis=-1)

        # Calculate confusion matrix and classification report
        conf_matrix = confusion_matrix(y_test_encoded, y_pred)
        class_report = classification_report(y_test_encoded, y_pred)

        print("Confusion Matrix:")
        print(conf_matrix)
        print("\nClassification Report:")
        print(class_report)

    else:
        print("Error: Dataset contains no samples.")
else:
    print("Error: Dataset directory does not exist.")


Loaded image: C:/Users/Maimoona Sabir/Desktop/CV\Buildings2\b1.jpg, Label: Buildings2, Shape: (224, 224)
Loaded image: C:/Users/Maimoona Sabir/Desktop/CV\Buildings2\b11.jpg, Label: Buildings2, Shape: (224, 224)
Loaded image: C:/Users/Maimoona Sabir/Desktop/CV\Buildings2\b12.jpg, Label: Buildings2, Shape: (224, 224)
Loaded image: C:/Users/Maimoona Sabir/Desktop/CV\Buildings2\b13.jpg, Label: Buildings2, Shape: (224, 224)
Loaded image: C:/Users/Maimoona Sabir/Desktop/CV\Buildings2\b14.jpg, Label: Buildings2, Shape: (224, 224)
Loaded image: C:/Users/Maimoona Sabir/Desktop/CV\Buildings2\b15.jpg, Label: Buildings2, Shape: (224, 224)
Loaded image: C:/Users/Maimoona Sabir/Desktop/CV\Buildings2\b2.jpg, Label: Buildings2, Shape: (224, 224)
Loaded image: C:/Users/Maimoona Sabir/Desktop/CV\Buildings2\b3.jpg, Label: Buildings2, Shape: (224, 224)
Loaded image: C:/Users/Maimoona Sabir/Desktop/CV\Buildings2\b4.jpg, Label: Buildings2, Shape: (224, 224)
Loaded image: C:/Users/Maimoona Sabir/Desktop/CV\B

c:\Users\Maimoona Sabir\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/6


c:\Users\Maimoona Sabir\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.2857 - loss: 8.5731 - val_accuracy: 0.1429 - val_loss: 396.1486
Epoch 2/6
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.1562 - loss: 391.6403 - val_accuracy: 0.4286 - val_loss: 74.6762
Epoch 3/6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.4286 - val_loss: 74.6762
Epoch 4/6


c:\Users\Maimoona Sabir\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.1905 - loss: 85.2502 - val_accuracy: 0.2143 - val_loss: 11.7855
Epoch 5/6
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.2188 - loss: 10.1168 - val_accuracy: 0.2143 - val_loss: 6.6073
Epoch 6/6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.2143 - val_loss: 6.6073
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step - accuracy: 0.2143 - loss: 6.6073
Test Loss: 6.607332706451416, Test Accuracy: 0.2142857164144516
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step
Confusion Matrix:
[[0 6 0 0]
 [0 3 0 0]
 [0 2 0 0]
 [0 3 0 0]]

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.21      1.00      0.35         3
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         3

    accuracy                           0.21        14
   macro avg       0.05      0.25      0.

c:\Users\Maimoona Sabir\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Maimoona Sabir\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Maimoona Sabir\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh

PREDICTION ON UNSEEN DATA

In [43]:
# Load and preprocess the unseen dataset
unseen_data_dir = 'C:/Users/Maimoona Sabir/Desktop/CV/Validation'
X_unseen, y_unseen = load_dataset(unseen_data_dir)

# Check if the unseen dataset contains enough samples
if len(X_unseen) > 0:
    # Reshape X_unseen to add the channel dimension
    X_unseen = np.expand_dims(X_unseen, axis=-1)  # Assuming grayscale images
    
    # Convert labels to numerical format using LabelEncoder
    y_unseen_encoded = label_encoder.transform(y_unseen)

    # Evaluate the model on the unseen dataset
    unseen_loss, unseen_accuracy = model.evaluate(X_unseen, y_unseen_encoded)
    print(f'Unseen Data Loss: {unseen_loss}, Unseen Data Accuracy: {unseen_accuracy}')
    
    # Generate predictions for the unseen data
    y_unseen_pred = np.argmax(model.predict(X_unseen), axis=-1)

    # Calculate confusion matrix and classification report for the unseen data
    unseen_conf_matrix = confusion_matrix(y_unseen_encoded, y_unseen_pred)
    unseen_class_report = classification_report(y_unseen_encoded, y_unseen_pred)

    print("Confusion Matrix for Unseen Data:")
    print(unseen_conf_matrix)
    print("\nClassification Report for Unseen Data:")
    print(unseen_class_report)
else:
    print("Error: Unseen dataset contains no samples.")


Loaded image: C:/Users/Maimoona Sabir/Desktop/CV/Validation\Buildings2\b1.jpg, Label: Buildings2, Shape: (224, 224)
Loaded image: C:/Users/Maimoona Sabir/Desktop/CV/Validation\Buildings2\b2.jpg, Label: Buildings2, Shape: (224, 224)
Loaded image: C:/Users/Maimoona Sabir/Desktop/CV/Validation\Buildings2\b3.jpg, Label: Buildings2, Shape: (224, 224)
Loaded image: C:/Users/Maimoona Sabir/Desktop/CV/Validation\Food2\f1.jpg, Label: Food2, Shape: (224, 224)
Loaded image: C:/Users/Maimoona Sabir/Desktop/CV/Validation\Food2\f2.jpg, Label: Food2, Shape: (224, 224)
Loaded image: C:/Users/Maimoona Sabir/Desktop/CV/Validation\Food2\f3.jpg, Label: Food2, Shape: (224, 224)
Loaded image: C:/Users/Maimoona Sabir/Desktop/CV/Validation\Other2\O1.jpg, Label: Other2, Shape: (224, 224)
Loaded image: C:/Users/Maimoona Sabir/Desktop/CV/Validation\Other2\O2.jpg, Label: Other2, Shape: (224, 224)
Loaded image: C:/Users/Maimoona Sabir/Desktop/CV/Validation\Other2\O3.jpg, Label: Other2, Shape: (224, 224)
Loaded ima

c:\Users\Maimoona Sabir\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Maimoona Sabir\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Maimoona Sabir\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh